In [1]:
import os
import numpy as np


In [2]:
import PyPDF2

def extract_pages(input_pdf, output_pdf, start_page, end_page):
    # Open the input PDF file
    with open(input_pdf, 'rb') as infile:
        reader = PyPDF2.PdfFileReader(infile)
        
        # Create a PDF writer object for the output file
        writer = PyPDF2.PdfFileWriter()
        
        # Add the specified pages to the writer object
        for page_num in range(start_page, end_page + 1):
            page = reader.getPage(page_num)
            writer.addPage(page)
        
        # Write the output PDF file
        with open(output_pdf, 'wb') as outfile:
            writer.write(outfile)

# Define the input and output PDF file names
input_pdf = '/home/pk/Desktop/gitmaster/PDSA/pyq/quiz1/Jan 2023.pdf'
output_pdf = 'jan 2023_NO_ans.pdf'

# Define the range of pages to extract (assuming the second file is a subset of the first)
# Adjust the page numbers as needed
start_page = 0  # First page (0-indexed)
end_page = 10   # Last page to extract (adjust as needed)

# Extract the specified pages and create the output PDF
extract_pages(input_pdf, output_pdf, start_page, end_page)

print(f"Extracted pages {start_page + 1} to {end_page + 1} from {input_pdf} and saved to {output_pdf}")

DeprecationError: PdfFileReader is deprecated and was removed in PyPDF2 3.0.0. Use PdfReader instead.

In [3]:
import PyPDF2

def extract_pages(input_pdf, output_pdf, start_page, end_page):
    # Open the input PDF file
    with open(input_pdf, 'rb') as infile:
        reader = PyPDF2.PdfReader(infile)
        
        # Create a PDF writer object for the output file
        writer = PyPDF2.PdfWriter()
        
        # Add the specified pages to the writer object
        for page_num in range(start_page, end_page + 1):
            page = reader.pages[page_num]
            # Remove answers marked with crosses, ticks, and color-coded text
            text = re.sub(r'$$X$$|$$√$$|$$✓$$', '', text)  # Remove crosses and ticks
            text = re.sub(r'$$(green|red)$$', '', text)  # Remove color-coded text

            writer.add_page(page)
        
        # Write the output PDF file
        with open(output_pdf, 'wb') as outfile:
            writer.write(outfile)

# Define the input and output PDF file names
input_pdf = '/home/pk/Desktop/gitmaster/PDSA/pyq/quiz1/Jan 2023.pdf'
output_pdf = 'jan 2023_NO_ans.pdf'

# Define the range of pages to extract (assuming the second file is a subset of the first)
# Adjust the page numbers as needed
start_page = 0  # First page (0-indexed)
end_page = 10   # Last page to extract (adjust as needed)

# Extract the specified pages and create the output PDF
extract_pages(input_pdf, output_pdf, start_page, end_page)

print(f"Extracted pages {start_page + 1} to {end_page + 1} from {input_pdf} and saved to {output_pdf}")

Extracted pages 1 to 11 from /home/pk/Desktop/gitmaster/PDSA/pyq/quiz1/Jan 2023.pdf and saved to jan 2023_NO_ans.pdf


In [5]:
import fitz  # PyMuPDF

def remove_marks_and_convert_to_grayscale(input_pdf, output_pdf):
    # Open the input PDF file
    document = fitz.open(input_pdf)
    
    # Iterate through each page
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        pix = page.get_pixmap()
        
        # Convert the page to grayscale
        pix = fitz.Pixmap(pix, 0)  # 0 means grayscale
        
        # Create a new page with the grayscale image
        new_page = document.new_page(width=pix.width, height=pix.height)
        new_page.insert_image(new_page.rect, pixmap=pix)
        
        # Remove the original page
        document.delete_page(page_num)
    
    # Save the modified document
    document.save(output_pdf)

# Define the input and output PDF file names
input_pdf = '/home/pk/Desktop/gitmaster/PDSA/pyq/quiz1/Jan 2023.pdf'
output_pdf = 'jan 2023_grayscale.pdf'

# Remove marks and convert to grayscale
remove_marks_and_convert_to_grayscale(input_pdf, output_pdf)

print(f"Processed {input_pdf} and saved to {output_pdf}")

Processed /home/pk/Desktop/gitmaster/PDSA/pyq/quiz1/Jan 2023.pdf and saved to jan 2023_grayscale.pdf


In [4]:
import PyPDF2
import re

# Open the first file (question paper with answers)
with open('/home/pk/Desktop/gitmaster/PDSA/pyq/quiz1/Jan 2023.pdf', 'rb') as f:
    pdf = PyPDF2.PdfReader(f)
    text = ''
    for page in pdf.pages:
        text += page.extract_text()

# Remove answers marked with crosses, ticks, and color-coded text
text = re.sub(r'$$X$$|$$√$$|$$✓$$', '', text)  # Remove crosses and ticks
text = re.sub(r'$$(green|red)$$', '', text)  # Remove color-coded text

# Split the text into individual questions
questions = re.split(r'\n\s*\n', text)

# Create the second file (question paper without answers)
with open('jan_2023_NO_ans.pdf', 'w') as f:
    for question in questions:
        f.write(question + '\n\n')

print("Second file generated: jan 2023_NO_ans.pdf")

Second file generated: jan 2023_NO_ans.pdf


In [6]:
import PyPDF2
import re

def extract_pages(input_pdf, output_pdf, start_page, end_page):
    # Open the input PDF file
    with open(input_pdf, 'rb') as infile:
        reader = PyPDF2.PdfReader(infile)
        
        # Create a PDF writer object for the output file
        writer = PyPDF2.PdfWriter()
        
        # Add the specified pages to the writer object
        for page_num in range(start_page, end_page + 1):
            page = reader.pages[page_num - 1]  # PyPDF2 pages are 0-indexed
            
            # Extract the text from the page
            text = ''
            if '/XObject' in page['/Resources']:
                xObject = page['/Resources']['/XObject'].get_object()
                for obj in xObject:
                    if xObject[obj]['/Subtype'] == '/Form':
                        stream = xObject[obj].getObject()['/Length'].getObject().decode('utf-8')
                        text += stream
            
            # Remove answers marked with crosses, ticks, and color-coded text
            text = re.sub(r'$$X$$|$$√$$|$$✓$$', '', text)  # Remove crosses and ticks
            text = re.sub(r'$$(green|red)$$', '', text)  # Remove color-coded text
            
            # Create a new page with the modified text
            new_page = writer.add_blank_page(width=page['/MediaBox'][2], height=page['/MediaBox'][3])
            new_page.merge_page(page)
            
        # Write the output PDF file
        with open(output_pdf, 'wb') as outfile:
            writer.write(outfile)

# Define the input and output PDF file names
input_pdf = '/home/pk/Desktop/gitmaster/PDSA/pyq/quiz1/Jan 2023.pdf'
output_pdf = 'jan 2023_NO_ans.pdf'

# Define the range of pages to extract (assuming the second file is a subset of the first)
# Adjust the page numbers as needed
start_page = 1  # First page (1-indexed)
end_page = 10   # Last page to extract (adjust as needed)

# Extract the specified pages and create the output PDF
extract_pages(input_pdf, output_pdf, start_page, end_page)

print(f"Extracted pages {start_page} to {end_page} from {input_pdf} and saved to {output_pdf}")

Extracted pages 1 to 10 from /home/pk/Desktop/gitmaster/PDSA/pyq/quiz1/Jan 2023.pdf and saved to jan 2023_NO_ans.pdf
